## Fetch Weather Data Online
Use a Web-API to fetch weather-data and send it to the Kafka topic.

**NOTE**: The API has rate-limits, so not overdo it! `Only for non-commercial use and less than 10.000 daily API calls. See Terms for more details.`

In [4]:
# Kafka defintions
KAFKA_BROKER = "kafka:9092"
TOPIC = "weather-data"

In [3]:
from confluent_kafka import Producer
import json
import time
import requests

# Initialize Kafka producer
producer = Producer({'bootstrap.servers': KAFKA_BROKER})

# Callback to confirm delivery
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed: {err}")
    else:
        print(f"Sent: {msg.value().decode('utf-8')}")

# Function to fetch weather data
def fetch_weather_data():
    url = 'https://api.open-meteo.com/v1/forecast'
    params = {
        'latitude': 47.72,
        'longitude': 13.09,
        'hourly': 'temperature_2m,relative_humidity_2m,wind_speed_10m',
        'timezone': 'UTC',
        'forecast_days': 1
    }
    response = requests.get(url, params=params)
    return response.json()

# Fetch and send data to Kafka
weather_data = fetch_weather_data()

## Publish to the Kafka topic 'weather-forecast'

In [3]:
hourly_data = weather_data['hourly']
for i, timestamp in enumerate(hourly_data['time']):
    record = {
        'timestamp': timestamp,
        'temperature': hourly_data['temperature_2m'][i],
        'humidity': hourly_data['relative_humidity_2m'][i],
        'wind_speed': hourly_data['wind_speed_10m'][i]
    }
    payload = json.dumps(record).encode('utf-8')
    producer.produce(TOPIC, value=payload, callback=delivery_report)
    producer.poll(0)  # Trigger delivery report callbacks
    time.sleep(1)  # Simulate real-time data streaming

producer.flush()

Sent: {"timestamp": "2025-04-25T00:00", "temperature": 8.8, "humidity": 88, "wind_speed": 2.9}
Sent: {"timestamp": "2025-04-25T01:00", "temperature": 8.8, "humidity": 86, "wind_speed": 1.8}
Sent: {"timestamp": "2025-04-25T02:00", "temperature": 8.5, "humidity": 88, "wind_speed": 1.1}
Sent: {"timestamp": "2025-04-25T03:00", "temperature": 8.1, "humidity": 89, "wind_speed": 2.9}
Sent: {"timestamp": "2025-04-25T04:00", "temperature": 8.1, "humidity": 89, "wind_speed": 2.3}
Sent: {"timestamp": "2025-04-25T05:00", "temperature": 8.3, "humidity": 85, "wind_speed": 0.8}
Sent: {"timestamp": "2025-04-25T06:00", "temperature": 8.6, "humidity": 96, "wind_speed": 0.5}
Sent: {"timestamp": "2025-04-25T07:00", "temperature": 9.5, "humidity": 92, "wind_speed": 2.1}
Sent: {"timestamp": "2025-04-25T08:00", "temperature": 10.6, "humidity": 81, "wind_speed": 1.1}
Sent: {"timestamp": "2025-04-25T09:00", "temperature": 10.5, "humidity": 81, "wind_speed": 2.3}
Sent: {"timestamp": "2025-04-25T10:00", "tempera

0

## Now consume the created values from the Kafka topic

In [ ]:
from confluent_kafka import Consumer, KafkaException
import json

# Initialize Kafka consumer
consumer_conf = {
    'bootstrap.servers': KAFKA_BROKER,
    'group.id': 'weather-forecast-group',
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(consumer_conf)

# Subscribe to the topic
consumer.subscribe([TOPIC])

print("Listening to topic:", TOPIC)

try:
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            value = msg.value().decode('utf-8')
            data = json.loads(value)
            print(f"Received: {json.dumps(data, indent=2)}")

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    consumer.close()

Listening to topic: weather-data
